# Cleaning 
This notebook reads and cleans the raw dataset. It then writes the cleaned data to a new csv.

In [1]:
import numpy as np
import pandas as pd

## Read data 
Column definitions:  
  
| Variable | Definition                                 | Key                                            |
|----------|--------------------------------------------|------------------------------------------------|
| survived | Survived                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex	   | Sex	                                    |                                                |
| Age	   | Age in years	                            |                                                |
| sibsp	   | # of siblings / spouses aboard the Titanic |	                                             |
| parch	   | # of parents / children aboard the Titanic	|                                                |
| ticket   | Ticket number	                            |                                                |
| fare	   | Passenger fare	                            |                                                |
| cabin	   | Cabin number                               |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

In [2]:
train = pd.read_csv("./data/raw/train.csv")
test = pd.read_csv("./data/raw/test.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Analysis for cleaning

### Identify columns with a high percentage of nulls
The output below shows that the columns Cabin, Age, and Embarked have null values. All other columns are complete

In [3]:
(train.isnull().sum()/train.shape[0]).sort_values(ascending=False)

Cabin          0.771044
Age            0.198653
Embarked       0.002245
Fare           0.000000
Ticket         0.000000
Parch          0.000000
SibSp          0.000000
Sex            0.000000
Name           0.000000
Pclass         0.000000
Survived       0.000000
PassengerId    0.000000
dtype: float64

## Cleaning and Feature engineering 
The following steps are applied to both the train and test datasets

In [4]:
# Create list of data sets so that the same transformations can be applied to both 
full_data = [train, test]

### Fill nulls

The Embarked column only had a small percentage of nulls and the below output shows that the majority of passangers embarked at Southampton so for now any null values will be set to Southampton

In [5]:
train.groupby(train["Embarked"]).size()

Embarked
C    168
Q     77
S    644
dtype: int64

In [6]:
for d in full_data:
    d["Embarked"] = d["Embarked"].fillna("S")

The Age colum has a more significant number of nulls so for now the nulls will be filld with a random value from that falls within 1 standard deviation of the mean age

In [7]:
for d in full_data:
    age_avg = d["Age"].mean()
    age_std = d["Age"].std()
    age_null_count = d["Age"].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    d["Age"][np.isnan(d["Age"])] = age_null_random_list
    d["Age"] = d["Age"].astype(int)

/home/treilly/Documents/projects/machine-learning-scratch/sklearn-titanic/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Map strings to ints
Convert the strings in the dataframe to intergers

In [8]:
for d in full_data:
    # Map Sex
    d["Sex"] = d["Sex"].map( {"female": 0, "male": 1} ).astype(int)
    # Mapping Embarked
    d["Embarked"] = d["Embarked"].map( {"S": 0, "C": 1, "Q": 2} ).astype(int)

### Drop unnessary columns
The PassangerId and Name columns are unlikley to add anything to the model on their own so they will be dropped.
The Ticket and Cabin columns will also be dropped for now as much of the same infomation could be gained from Pclass.

In [9]:
drop_elements = ["PassengerId", "Name", "Ticket", "Cabin"]
for d in full_data:
    d.drop(drop_elements, axis = 1, inplace=True)

In [14]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22,1,0,7.2500,0
1,1,1,0,38,1,0,71.2833,1
2,1,3,0,26,0,0,7.9250,0
3,1,1,0,35,1,0,53.1000,0
4,0,3,1,35,0,0,8.0500,0


## Write data

In [15]:
train.to_csv("./data/clean/train.csv", header=True, index=False)
test.to_csv("./data/clean/test.csv", header=True, index=False)